## Neighborhood weekly averages -- Map

#### NB setup

In [13]:
import pandas as pd
import json
import geopandas as gpd
ntimport plotly.express as px
import os

In [14]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# set directory explicitly
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

#### Reading in data

In [15]:
#read in csv file
df_nta = pd.read_csv(r'data\nycnta\NTA_wkpctaway_01-09-20.csv')

# melt for animation formatl
list_date = df_nta.columns.tolist()
list_date = list_date[1:] # all weeks
# list_date = list_date[61:63] # march 1st to march 2nd
df = pd.melt(df_nta, id_vars=['NTA'],value_vars = list_date)
df = df.rename(columns={'NTA':'id'}) #rename so the join ids are the same
df['variable'] = df.variable.str.replace('-','/')
df['val_pct'] = df['value']*100 # convert it into percentage
df.tail(20)

,id,variable,value,val_pct
7312,QN72,9/23/20,0.021850,2.185015
7313,QN76,9/23/20,0.026274,2.627434
7314,SI01,9/23/20,0.020595,2.059534
7315,SI05,9/23/20,0.018503,1.850289
7316,SI07,9/23/20,0.014498,1.449809
7317,SI08,9/23/20,0.018909,1.890934
7318,SI11,9/23/20,0.014288,1.428754
7319,SI12,9/23/20,0.018170,1.817040
7320,SI14,9/23/20,0.014465,1.446469
7321,SI22,9/23/20,0.022710,2.270980


#### Read in spatial files

In [16]:
#read in geojson file
map_ntas = json.load(open(r'data\shapefiles\NTA map.geojson','r'))
#check properties, ensure proper file
map_ntas['features'][0]['properties']

{'ntacode': 'BK43',
 'shape_area': '35799637.8103',
 'ntaname': 'Midwood',
 'shape_leng': '27996.5912736',
 'boroname': 'Brooklyn',
 'borocode': '3',
 'countyfips': '047'}

In [17]:
# reading in NBHOOD shapefile
nbhood = gpd.read_file(r'data\shapefiles\nynta.shp')
# drop unnecessary columns
nbhood.drop(columns = ['BoroCode','CountyFIPS','Shape_Leng','Shape_Area','geometry'])
# join on NTAcode
df = pd.merge(df, nbhood, left_on = 'id', right_on = 'NTACode', how = 'inner')
# rename columns for viz
df = df.rename({'NTAName':'neighborhood', 'variable':'Week of ', 'val_pct':'percent_away'}, axis = 1)
df['neighborhood'] = df['neighborhood'].str.replace('-',' ') # replace - value with space
df.head()

,id,Week of,value,percent_away,BoroCode,BoroName,CountyFIPS,NTACode,neighborhood,Shape_Leng,Shape_Area,geometry
0,BK09,1/1/20,0.031373,3.137318,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
1,BK09,1/8/20,0.025132,2.513235,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
2,BK09,1/15/20,0.028373,2.837318,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
3,BK09,1/22/20,0.017018,1.701767,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."
4,BK09,1/29/20,0.015384,1.538359,3,Brooklyn,047,BK09,Brooklyn Heights Cobble Hill,14264.786528,9.983620e+06,"POLYGON ((986367.735 190549.239, 985813.835 19..."


#### Plotly visualization

In [18]:
fig = px.choropleth_mapbox(df, geojson = map_ntas, # specify your dataframe and spatial file (GeoJSON)
                   locations ='id', # specify location
                   featureidkey = 'properties.ntacode',
                   color = 'percent_away', # influences color
                   hover_name = 'neighborhood',
                   animation_frame ='Week of ', # animate by date column
                   color_continuous_scale = "oranges",
                   opacity = 0.99,
                   labels={'percent_away':'% of Residents', 'id':'NTA Code'},
                   range_color = [0,30])
# fig.add_annotation(text = 'Percent of NYC Residents<br>Away from Home >22 hours<br>per Day by Neighborhood',
#                   align = 'left', x = 0.03, y = 0.95,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 21 ))
#fig.add_annotation(text = 'Residents Away from Home',
 #                 align = 'right', x = 0.08, y = 0.90,
  #                showarrow = False,
   #               bordercolor = None,
    #              bgcolor = None,
     #             opacity = 0.95,
      #            font = dict(family = "Droid Serif", color = 'black', size = 28 ))
# fig.add_annotation(text = 'March 1, 2020 to May 31st, 2020',
#                   align = 'right', x = 0.03, y = 0.82,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 17 ))
# fig.add_annotation(text = "Click 'Play' to run through the animation<br>Or drag the slider to view specific dates",
#                   align = 'left', x = 0.73, y = 0.02,
#                   showarrow = False,
#                   bordercolor = None,
#                   bgcolor = None,
#                   opacity = 0.95,
#                   font = dict(family = "Arial", color = 'white', size = 12 ))
fig.update_layout(mapbox_zoom = 9.45,
                 mapbox_center = {'lat':40.7, 'lon':-73.95},
                 mapbox_style="carto-darkmatter",
                 width = 950,
                 height = 780,
       #        legend_title_text = '% of NTA Residents',
                 legend = dict(yanchor = "top", y = 0.6, xanchor = "left", x = 0.10))

#fig.show()
fig.write_html(r'Maps\html_maps\pct_time_away.html', auto_open = True, auto_play = False, include_plotlyjs = 'cdn')
# with open('pct_time_away.html') as f:
#     f.write(fig.to_html(auto_play = False, include_plotlyjs = 'cdn'))